# GeoIT3D_to_glTF - Colab
Notebook to convert a GeoIT3D ZIP into glTF/GLB directly in Colab.
- Clean workspace, clone repo, install in editable mode.
- Upload ZIP (required) and ISO/AGID .xlsx (optional).
- Paths are set automatically from uploaded files.


In [ ]:
# ### Workspace setup (clean + clone)
import os, shutil, subprocess
base_path = '/content'
repo_name = 'GeoIT3D_to_glTF'
repo_path = os.path.join(base_path, repo_name)

def clean_repo_directory():
    if os.path.exists(repo_path):
        shutil.rmtree(repo_path)
        print(f'Removed {repo_path}')

clean_repo_directory()
os.chdir(base_path)
subprocess.run(['git', 'clone', 'https://github.com/BaterHub/GeoIT3D_to_glTF.git'], check=True)
os.chdir(repo_path)
print('Working dir:', os.getcwd())


In [ ]:
!pip install -q -e .


In [ ]:
# Upload files from local machine (ZIP required, ISO/AGID optional)
from google.colab import files
uploaded = files.upload()
print('Uploaded:', list(uploaded.keys()))


In [ ]:
from pathlib import Path

# After clone we are already in /content/GeoIT3D_to_glTF
BASE = Path.cwd()
REPO = Path.cwd()

uploaded_names = list(uploaded.keys())
if not uploaded_names:
    raise ValueError('Upload at least the GeoIT3D ZIP (.zip).')

def pick_by_ext(exts):
    for name in uploaded_names:
        for ext in exts:
            if name.lower().endswith(ext):
                return name
    return None

zip_name = pick_by_ext(['.zip'])
if zip_name is None:
    raise ValueError('No ZIP found among uploaded files (expected .zip).')
iso_name = pick_by_ext(['.xlsx', '.xlsm'])

# files.upload() saves files into current working dir
ZIP_PATH = BASE / zip_name
ISO_SHEET = (BASE / iso_name) if iso_name else None
OUTPUT_DIR = REPO / 'output' / ZIP_PATH.stem

print('Working dir (where files are saved):', BASE)
print('ZIP:', ZIP_PATH, 'exists:', ZIP_PATH.exists())
print('ISO sheet:', ISO_SHEET, 'exists:', ISO_SHEET.exists() if ISO_SHEET else False)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print('Output dir:', OUTPUT_DIR)


In [ ]:
import subprocess, shlex
cmd = ['geoit3d-to-gltf', ZIP_PATH.as_posix(), '--output-dir', OUTPUT_DIR.as_posix()]
if ISO_SHEET:
    cmd += ['--iso-sheet', ISO_SHEET.as_posix()]
print('Running:', ' '.join(shlex.quote(c) for c in cmd))
subprocess.run(cmd, check=True)
